In [2]:
from datasets import load_dataset,load_from_disk

dataset = load_from_disk("/llm_data/yellp")

/root/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [5]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

,label,text
0,5 stars,"Love this place. They are so friendly and very helpful when you come in. They ensure you have everything you need and more. And then there is the price. WHAT A DEAL. We have one large, hairy dog and having a nice big tub to wash him in, get all that hair brushed out and get his teeth, ears and hair blow dried is such a treat!"
1,1 star,"Stupid Idiots left a flyer on my door, but after ordering called back 10 minutes later and said it was to far to drive. WTF"
2,1 star,"First and foremost, the hibachi grill was fantastic however it was not worth an hour wait at the grill. We were seated 30 minutes after our reservation time and then waited another 45 minutes for the chef to come out. Overall, the service was horrific."
3,4 stars,"Great place on Mill Ave for happy hour or a casual work lunch. Bruschetta plate is always delicious and allows you to choose a nice mix of toppings. A couple of my favorites are the smoked prosciutto, marscapone & fig and the roasted artichoke, sauteed spinach & red pepper.\n\nWine is reasonably priced (and discounted for happy hour), and the salads are tasty. The balsamic drizzle on the caprese salad was thick and sweet - much higher quality than you'd find at most restaurants.\n\nOutdoor seating is a great option when it's not too hot out, and service has always been prompt and courteous. Overall, a good option when you are looking for a step up in quality from most of the other restaurants on Mill."
4,1 star,"I was craving pasta and I wanted a quick fix, so I decided to come here. Biggest mistake ever. \n\nI ordered the Broccoli Pesto with angel hair pasta. Absolutely disgusting. It tasted like I was just taking bites out of raw garlic cloves. The garlic was so strong, it literally made my eyes water. \n\nThe pasta was cooked fine and the chicken was okay, if only I was able to actually taste the broccoli pesto, and not just clumps of garlic.\n\nThis hasn't been my first time here. I've order a few other times, and everything else that I've ordered in the past has always been bland, dry, and tasteless. \n\nAfter this experience, I probably will never go back...Unless my pasta craving is a do or die situation. I'll just have to learn how to be patient and cook my own pasta from now on."
5,1 star,"Food was extremely disappointing! The \""prime cut\"" steak although not overcooked was the toughest steak I have ever tried. Service was not the best either,"
6,2 star,"I have eaten here before, numerous times. But it's been several years. Last weekend we decided to give it a try. While I love the atmosphere, I was highly disappointed in the food. Way smaller portions and way more expensive than I remember. We also got duped into buying the glasses our cocktails came in. The menu says you can pay like an extra three or four dollars per cocktail to keep the glass, or you can choose not to. We said we didn't want the glasses, but when our bill came we realized they'd charged us for them anyway. Our waiter made up a lame story about the price that we knew was a fib because we read the menu (which he'd conveniently removed from the table). Finally, rather than arguing over a few dollars, we took the glasses and left. It was $120 plus tip for just my husband and me to eat there (and we are not large people)."
7,4 stars,"Wonderful, comfortably cheery atmosphere. It's so warm and cozy to sit here on a winter night with a hot beverage and a book, or a cheese plate and Scrabble with my husband. The drinks are wonderful; I haven't really had anything here I didn't like. In addition to the usual coffee/tea concoctions, they have juices and a few alcoholic beverages. The food is also quite good, unlike a lot of other coffee shops I've tried. Try the quesadilla or the Aroma Salad. Good music, too! My only beef with them is that they close super-early in the evenings during semester break, meaning that those of us who actually LIVE here don't get to partake while the students are gone. I would unde

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [50]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/llm_model/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=128)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:10<00:00, 4962.22 examples/s]


In [28]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"If you go to Cave Creek you must go to the Buffalo Chip, Harold's , hideaway etc. just great local restaurants in a great town north of Phoenix/Scottsdale. I like the Chip more for drinking dancing and rodeo than the food. But it is okay for a little grub But it is fun. So try it","[101, 1409, 1128, 1301, 1106, 11900, 3063, 1128, 1538, 1301, 1106, 1103, 6911, 20379, 117, 6587, 112, 188, 117, 4750, 7138, 3576, 119, 1198, 1632, 1469, 7724, 1107, 170, 1632, 1411, 1564, 1104, 6343, 120, 2796, 20537, 119, 146, 1176, 1103, 20379, 1167, 1111, 5464, 5923, 1105, 8335, 1186, 1190, 1103, 2094, 119, 1252, 1122, 1110, 3008, 1111, 170, 1376, 176, 5082, 1830, 1252, 1122, 1110, 4106, 119, 1573, 2222, 1122, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [60]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(300000))
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [61]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/llm_model/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /llm_model/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [62]:
from transformers import TrainingArguments

model_dir = "/llm_model/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [63]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=

### 训练过程中的指标评估（Evaluate)

In [64]:
import numpy as np
import evaluate

metric = evaluate.load("/llm_model/evaluate-main/metrics/accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [65]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [66]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [67]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.986600,0.925905,0.601580
2,0.850300,0.893299,0.620360
3,0.777000,0.898587,0.626900


TrainOutput(global_step=112500, training_loss=0.8747206936984592, metrics={'train_runtime': 22606.5292, 'train_samples_per_second': 39.812, 'train_steps_per_second': 4.976, 'total_flos': 5.92015820544e+16, 'train_loss': 0.8747206936984592, 'epoch': 3.0})

In [69]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(50000))

In [70]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.8985869288444519,
 'eval_accuracy': 0.6269,
 'eval_runtime': 337.2336,
 'eval_samples_per_second': 148.265,
 'eval_steps_per_second': 18.533,
 'epoch': 3.0}

In [71]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [72]:
trainer.save_state()